In [1]:
%load_ext autoreload

%autoreload 2


In [2]:
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import yfinance as yf
from tqdm import tqdm

while 'notebooks' in os.getcwd():
    os.chdir("..")
    
from src.preprocessing.finbert.get_news import get_news, \
    filter_news_with_name, save_company_news_df,\
    read_company_news_df
from src.preprocessing.finbert.sentiment import generate_sentiment_from_title

api_key = '64d11b117233d0.77790833'

In [3]:
from transformers import BertTokenizer, BertForSequenceClassification

finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

2023-08-19 18:15:08.817678: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-19 18:15:08.862430: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-19 18:15:08.863048: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-19 18:15:09.743028: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Data acquisition

In [5]:
start_year = 2019
end_year = 2023
stock = "AMZN"

amzn_news, amzn_complete = get_news(stock)

100%|██████████| 12/12 [00:44<00:00,  3.68s/it]


In [6]:
len(amzn_complete)

19451

In [7]:
news_with_amzn = filter_news_with_name(amzn_complete, 'Amazon')

In [8]:
len(news_with_amzn)

7062

In [10]:
sentiment_amzn = generate_sentiment_from_title(
    news_with_amzn,
    tokenizer,
    finbert
)

100%|██████████| 7062/7062 [08:19<00:00, 14.14it/s]


In [11]:
amzn_df = pd.DataFrame(news_with_amzn)[["date","title","content"]]

In [12]:
amzn_df["sentiment"] = sentiment_amzn

In [13]:
save_company_news_df(amzn_df,
                     "amazon")

## Price analysis

In [14]:
amzn_df = read_company_news_df("amazon")

In [15]:
amzn_prices = yf.download(
    'AMZN',
    start='2019-12-30',
    end='2022-12-30'
)

[*********************100%***********************]  1 of 1 completed


In [16]:
amzn_prices.index = pd.to_datetime(amzn_prices.index)

In [17]:
amzn_prices

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-12-30,93.699997,94.199997,92.030998,92.344498,92.344498,73494000
2019-12-31,92.099998,92.663002,91.611504,92.391998,92.391998,50130000
2020-01-02,93.750000,94.900497,93.207497,94.900497,94.900497,80580000
2020-01-03,93.224998,94.309998,93.224998,93.748497,93.748497,75288000
2020-01-06,93.000000,95.184502,93.000000,95.143997,95.143997,81236000
...,...,...,...,...,...,...
2022-12-22,85.519997,85.680000,82.250000,83.790001,83.790001,81431300
2022-12-23,83.250000,85.779999,82.930000,85.250000,85.250000,57433700
2022-12-27,84.970001,85.349998,83.000000,83.040001,83.040001,57284000


In [19]:
amzn_df["date"] = pd.to_datetime(amzn_df["date"])
amzn_df["date"] = amzn_df["date"].dt.date

In [20]:
amzn_df = amzn_df\
    .drop_duplicates(subset = "title")

In [21]:
amzn_df["sentiment_count"] = amzn_df["sentiment"].map({
    "neutral": 0 ,
    "positive" : 1,
    "negative" : -1
})

In [22]:
amzn_sentiment = amzn_df[["sentiment_count", "date"]]\
    .groupby("date")\
    .mean()

In [24]:
amzn_sentiment

,sentiment_count
date,
2019-02-27,0.000000
2019-03-11,0.000000
2019-04-15,0.000000
2019-07-16,0.000000
2019-08-28,0.000000
...,...
2022-12-27,0.111111
2022-12-28,0.000000
2022-12-29,0.000000


In [ ]:
amzn_sentiment.index = pd.to_datetime(amzn_sentiment.index)